<a href="https://colab.research.google.com/github/hemantmehra/ml_pytorch/blob/master/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [3]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 5

In [6]:
train_dataset = datasets.MNIST(root='/tmp', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='/tmp', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw


Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw



Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw


Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
cr = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=lr)

In [13]:
for epoch in range(num_epochs):
  loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
  for batch_idx, (data, targets) in loop:
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)
    scores = model(data)
    loss = cr(scores, targets)
    opt.zero_grad()
    loss.backward()
    opt.step()

    loop.set_description(f'Epoch [{epoch}/{num_epochs}]')

In [14]:
def check_acc(loader, model):
  num_c = 0
  num_s = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, pred = scores.max(1)
      num_c += (pred == y).sum()
      num_s += pred.size(0)

    print(float(num_c)/float(num_s))

In [15]:
check_acc(train_loader, model)
check_acc(test_loader, model)

0.9719833333333333
0.9622
